In [ ]:
import pandas as pd

df = pd.DataFrame({"A": [1, 2], "B": [3, 4]})

p = df.itertuples()
for i in p:
    print(type(i))


In [63]:
md = pd.read_feather("./movies_final.feather")
md.columns


Index(['url', 'Name', 'Poster', 'Genres', 'Actors', 'Director', 'Description',
       'DatePublished', 'Keywords', 'RatingCount', 'RatingValue',
       'ReviewAuthor', 'ReviewDate', 'ReviewBody', 'DescriptionLemma',
       'votewt'],
      dtype='object')

In [68]:
def get_genre_movies(genre):
    if genre == "All":
        return tuple(
            md[["Name", "votewt"]].itertuples(
                index=False,
            )
        )
        # return md[["Name", "votewt"]].itertuples(index=False, name=None)
    return tuple(
        md[md["Genres"].apply(lambda x: genre in x)][["Name", "votewt"]].iterrows()
    )
    # return md[md["Genres"].astype("str").str.contains(genre)][
    #     ["Name", "votewt"]
    # ].itertuples(index=False, name=None)


x = get_genre_movies("All")
x[0].Name


'Planet Earth II'

In [59]:
from pathlib import Path
import pandas as pd, numpy as np, joblib
from sklearn.feature_extraction.text import (
    TfidfVectorizer,
    HashingVectorizer,
)
from collections import Counter
import chunkdot.cosine_similarity_top_k as cstk

# New Preprocessed Dataset
md = pd.read_feather("./movies_final.feather")

feats = ["Genres", "Actors", "Keywords", "Name", "Director", "DescriptionLemma"]

combined_feats = ""
combined_feats += md["Genres"].apply(lambda x: " ".join(i for i in x) + " ") * 2
combined_feats += md["Keywords"].apply(lambda x: " ".join(i for i in x) + " ") * 4
combined_feats += md["Actors"].apply(
    lambda x: " ".join(i.replace(" ", "") for i in x) + " "
)
combined_feats += md["Name"].apply(lambda x: x.replace(" ", "") + " ") * 2
combined_feats += md["Director"].apply(lambda x: x.replace(" ", "") + " ") * 2
combined_feats += (md["DescriptionLemma"] + " ") * 7
# combined_feats+=md['Genres'].apply(lambda x:' '.join(i.replace(' ','') for i in x)+' ')*2
# combined_feats+=md['Keywords'].apply(lambda x:' '.join(i.replace(' ','') for i in x)+' ')*4
